# 1a

In [67]:
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Input
from sklearn.model_selection import train_test_split
from tensorflow.keras import backend as K

np.random.seed(42)

In [68]:
X = np.load("data/A1_data_75/images.npy")
y = np.load("data/A1_data_75/labels.npy")

X = X / 255

print(X.shape)
print(y.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=42)

print(X_train.shape, X_val.shape, X_test.shape)

(18000, 75, 75)
(18000, 2)
(10125, 75, 75) (3375, 75, 75) (4500, 75, 75)


In [69]:
batch_size = 128
num_classes = 24
epochs = 12

img_rows, img_cols = X_train.shape[1], X_train.shape[2]
input_shape = (img_rows, img_cols, 1)

if K.image_data_format() == 'channels_first':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_val = X_val.reshape(X_val.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_val = X_val.reshape(X_val.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

print(X_train.shape, X_val.shape, X_test.shape)

(10125, 75, 75, 1) (3375, 75, 75, 1) (4500, 75, 75, 1)


In [70]:
def to_categorical(y, num_classes):
    """
    Convert time to categorical labels.

    :param y: numpy array of shape (num_samples, 2) where the first column is hour (0-11)
              and the second column is minute (0-59)
    :param num_classes: total number of classes
    :return: numpy array with one-hot encoded labels
    """
    # check if it is already one-hot encoded
    if y.shape[1] == num_classes:
        return y

    class_ = y[:, 0]*2 + y[:, 1]//30

    return keras.utils.to_categorical(class_, num_classes)

# Convert labels to one-hot encoding
y_train = to_categorical(y_train, num_classes)
y_val = to_categorical(y_val, num_classes)
y_test = to_categorical(y_test, num_classes)

print(y_train.shape, y_val.shape, y_test.shape)

(10125, 24) (3375, 24) (4500, 24)


In [ ]:
def common_sense_categories(y_true, y_pred):
    """
    Custom accuracy metric

    :param y_true: true labels
    :param y_pred: predicted labels
    :return: accuracy score
    """
    true_class = np.argmax(y_true)
    pred_class = np.argmax(y_pred)
    
    acc = np.ceil(num_classes/2)-min(abs(true_class - pred_class), abs(abs(true_class - pred_class) - num_classes))
    return acc

In [71]:
model = Sequential([
    Input(shape=input_shape),
    Conv2D(32, kernel_size=(3, 3), activation='relu'),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.summary()

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_val, y_val))

score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 73, 73, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 71, 71, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 35, 35, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 35, 35, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 78400)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │    10,035,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 24)             │         3,096 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,057,240 (38.37 MB)

 Trainable params: 10,057,240 (38.37 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/12
80/80 ━━━━━━━━━━━━━━━━━━━━ 60s 704ms/step - accuracy: 0.0406 - loss: 3.1862 - val_accuracy: 0.0356 - val_loss: 3.1807
Epoch 2/12
80/80 ━━━━━━━━━━━━━━━━━━━━ 52s 650ms/step - accuracy: 0.0414 - loss: 3.1835 - val_accuracy: 0.0385 - val_loss: 3.1801
Epoch 3/12
80/80 ━━━━━━━━━━━━━━━━━━━━ 53s 662ms/step - accuracy: 0.0386 - loss: 3.1818 - val_accuracy: 0.0397 - val_loss: 3.1800
Epoch 4/12
80/80 ━━━━━━━━━━━━━━━━━━━━ 55s 686ms/step - accuracy: 0.0427 - loss: 3.1812 - val_accuracy: 0.0439 - val_loss: 3.1798
Epoch 5/12
80/80 ━━━━━━━━━━━━━━━━━━━━ 54s 679ms/step - accuracy: 0.0415 - loss: 3.1802 - val_accuracy: 0.0409 - val_loss: 3.1797
Epoch 6/12
80/80 ━━━━━━━━━━━━━━━━━━━━ 55s 682ms/step - accuracy: 0.0432 - loss: 3.1797 - val_accuracy: 0.0400 - val_loss: 3.1795
Epoch 7/12
80/80 ━━━━━━━━━━━━━━━━━━━━ 52s 650ms/step - accuracy: 0.0441 - loss: 3.1783 - val_accuracy: 0.0376 - val_loss: 3.1791
Epoch 8/12
80/80 ━━━━━━━━━━━━━━━━━━━━ 55s 689ms/step - accuracy: 0.0419 - loss: 3.1795 - val_accu